## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

TDC very fine, every tick is 25/30 nano seconds exactly, very precise; everything is like a clock, every cycle of 30 increments BX (like minuts for seconds); same goes for BX, between 0 and boh; after maximum ORBIT goes up... there are some conversions to be done, but roughly speaking is a clock: play vectorially with it!

In [ ]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [33]:
# -1 import things
import pandas as pd
import numpy as np
from numpy import random as npr
#import datetime as dt

In [20]:
# 0 find data file
data_file = !find ~/*/data_000637.txt
data_file = data_file[0]
print (data_file)

/home/pieripol/data/data_000637.txt


In [63]:
# 0.5 chose how much rows to read
# how many rows are there
total_rows = !grep -c -v "^#" $data_file
total_rows = int(total_rows[0])
min_rows = 10000
# how mutch rows to write
ROWS = npr.randint(min_rows, total_rows)
print(ROWS)

345028


In [64]:
# 1
data_frame = pd.read_csv(data_file, delimiter=',', nrows=ROWS)
data_frame

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
5,1,0,63,3869200167,2762,4
6,1,0,61,3869200167,2772,14
7,1,0,139,3869200167,2776,0
8,1,0,62,3869200167,2774,21
9,1,0,60,3869200167,2788,7


In [36]:
# max values are 3563 for BX_COUNTER and 29 for TDC_MEAS;
# it is safe to assume a range [0; 3563] for BX_COUNTER, a total of 3564 cycles
BX_X = 3564
TDC_X = 30
TDC_ns = 25./30.
ORBIT_min = data_frame['ORBIT_CNT'][0]
data_frame.describe()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
count,802693.0,802693.000000,802693.000000,8.026930e+05,802693.000000,802693.000000
mean,1.0,0.295713,55.877109,3.869204e+09,1788.694537,13.334688
std,0.0,0.456363,39.047411,2.257070e+03,1028.013015,9.203730
min,1.0,0.000000,1.000000,3.869200e+09,0.000000,0.000000
25%,1.0,0.000000,26.000000,3.869202e+09,902.000000,5.000000
50%,1.0,0.000000,57.000000,3.869204e+09,1791.000000,13.000000
75%,1.0,1.000000,64.000000,3.869206e+09,2681.000000,21.000000
max,1.0,1.000000,139.000000,3.869208e+09,3563.000000,29.000000


In [75]:
# 3
all_data = pd.read_csv(data_file, delimiter=',', nrows=(total_rows-1))
time_ns = ((all_data['ORBIT_CNT'].iloc[total_rows-2] - all_data['ORBIT_CNT'].iloc[0] )*BX_X*TDC_X + (all_data['BX_COUNTER'].iloc[total_rows-2] - all_data['BX_COUNTER'].iloc[0] )*TDC_X + (all_data['TDC_MEAS'].iloc[total_rows-2] - all_data['TDC_MEAS'].iloc[0] ))*TDC_ns
print( f"{total_rows-1} measures took: {time_ns:.0f} ns, or {time_ns//1000000000:.0f}s {(time_ns%1000000000)//1000000:.0f}ms {(time_ns%1000000)//1000:.0f}micros {(time_ns%1000):.0f}ns" )

1310720 measures took: 980416093 ns, or 0s 980ms 416micros 93ns


In [74]:
# 4
time_offset = data_frame['ORBIT_CNT'].iloc[0]*BX_X*TDC_X + data_frame['BX_COUNTER'].iloc[0]*TDC_X + data_frame['TDC_MEAS'].iloc[0]
data_frame['TIME_NS'] = pd.Series((((data_frame['ORBIT_CNT']*BX_X*TDC_X + data_frame['BX_COUNTER']*TDC_X + data_frame['TDC_MEAS']) - time_offset) * TDC_ns ), index=data_frame.index)
data_frame

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_NS
0,1,0,123,3869200167,2374,26,0.000000e+00
1,1,0,124,3869200167,2374,27,8.333333e-01
2,1,0,63,3869200167,2553,28,4.476667e+03
3,1,0,64,3869200167,2558,19,4.594167e+03
4,1,0,64,3869200167,2760,25,9.649167e+03
5,1,0,63,3869200167,2762,4,9.681667e+03
6,1,0,61,3869200167,2772,14,9.940000e+03
7,1,0,139,3869200167,2776,0,1.002833e+04
8,1,0,62,3869200167,2774,21,9.995833e+03
9,1,0,60,3869200167,2788,7,1.033417e+04


In [77]:
# 4 control
#time_offset = all_data['ORBIT_CNT'].iloc[0]*BX_X*TDC_X + all_data['BX_COUNTER'].iloc[0]*TDC_X + all_data['TDC_MEAS'].iloc[0]
all_data['TIME_NS'] = pd.Series((((all_data['ORBIT_CNT']*BX_X*TDC_X + all_data['BX_COUNTER']*TDC_X + all_data['TDC_MEAS']) - time_offset) * TDC_ns ), index=all_data.index)
all_data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_NS
0,1,0,123,3869200167,2374,26,0.000000e+00
1,1,0,124,3869200167,2374,27,8.333333e-01
2,1,0,63,3869200167,2553,28,4.476667e+03
3,1,0,64,3869200167,2558,19,4.594167e+03
4,1,0,64,3869200167,2760,25,9.649167e+03
5,1,0,63,3869200167,2762,4,9.681667e+03
6,1,0,61,3869200167,2772,14,9.940000e+03
7,1,0,139,3869200167,2776,0,1.002833e+04
8,1,0,62,3869200167,2774,21,9.995833e+03
9,1,0,60,3869200167,2788,7,1.033417e+04


In [82]:
data_frame['HEAD'] = pd.Series(npr.randint(0,2,ROWS), index=data_frame.index)
data_frame

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_NS
0,0,0,123,3869200167,2374,26,0.000000e+00
1,0,0,124,3869200167,2374,27,8.333333e-01
2,1,0,63,3869200167,2553,28,4.476667e+03
3,0,0,64,3869200167,2558,19,4.594167e+03
4,0,0,64,3869200167,2760,25,9.649167e+03
5,1,0,63,3869200167,2762,4,9.681667e+03
6,1,0,61,3869200167,2772,14,9.940000e+03
7,0,0,139,3869200167,2776,0,1.002833e+04
8,1,0,62,3869200167,2774,21,9.995833e+03
9,0,0,60,3869200167,2788,7,1.033417e+04


In [86]:
data_chosen = data_frame[data_frame['HEAD'] > 0]
data_chosen

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_NS
2,1,0,63,3869200167,2553,28,4.476667e+03
5,1,0,63,3869200167,2762,4,9.681667e+03
6,1,0,61,3869200167,2772,14,9.940000e+03
8,1,0,62,3869200167,2774,21,9.995833e+03
11,1,0,64,3869200167,2786,19,1.029417e+04
13,1,0,36,3869200167,2791,23,1.042250e+04
16,1,1,8,3869200167,2787,14,1.031500e+04
17,1,0,63,3869200167,2790,10,1.038667e+04
24,1,0,58,3869200167,2795,19,1.051917e+04
26,1,0,59,3869200167,2799,14,1.061500e+04
